# PermutationExplainer - inner working

In this notebook, we take a look at the inner working of the `PermutationExplainer` to understand how it calculates the SHAP  values. 

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier

import shap

## Load dataset

In [2]:
# Load iris dataset

X, y = load_iris(return_X_y=True, as_frame=True)

X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

X_train.shape, X_test.shape

((75, 4), (75, 4))

## Train the model to explain

In [4]:
gbm = XGBClassifier(random_state=3)

gbm.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [5]:
gbm.score(X_test, y_test)

0.9333333333333333

## Under the hood of the permutation explainer

In [6]:
# set up an explainer

# pass train set with 1 sample to keep it simple
exp = shap.PermutationExplainer(gbm.predict, X_train.head(1))

In [7]:
# this is the class that does the masking

from shap.utils._masked_model import MaskedModel

# the point to explain
sample = np.array([10, 20, 30, 40])

# the function that makes the explaining

fm = MaskedModel(exp.model, exp.masker, exp.link, exp.linearize_link, sample)

In [8]:
inds = fm.varying_inputs()
inds

array([0, 1, 2, 3], dtype=int64)

In [9]:
inds_mask = np.zeros(len(fm), dtype=bool)
inds_mask[inds] = True

inds_mask

array([ True,  True,  True,  True])

In [9]:
masks = np.zeros(2*len(inds)+1, dtype=int)
masks[0] = MaskedModel.delta_mask_noop_value

masks

array([2147483647,          0,          0,          0,          0,
                0,          0,          0,          0])

In [35]:
npermutations = 1

mi = []

for _ in range(npermutations):

    np.random.shuffle(inds)
    print(inds)
    i = 1
    
    for ind in inds:
        masks[i] = ind
        i += 1
    print(masks)
    
    for ind in inds:
        masks[i] = ind
        i += 1
    print(masks)
    
#     masked_inputs, varying_rows = fm.masker(masks, sample)
#     mi.append(masked_inputs)

[0 2 1 3]
[2147483647          0          2          1          3          0
          3          2          1]
[2147483647          0          2          1          3          0
          2          1          3]


In [36]:
masked_inputs, varying_rows = fm.masker(masks, sample)

mi.append(masked_inputs)

In [18]:
# the sample we are using to create
# the coalitions

X_train.head(1)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
3,4.6,3.1,1.5,0.2


In [19]:
# permutation 1

inds

array([1, 2, 3, 0], dtype=int64)

- empty -> 1
- 1 -> 2
- 12 -> 3
- 123 -> 0


- empty -> 0
- 0 -> 3
- 03 -> 2
- 023 -> 1

In [20]:
mi[0]

(   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
 0                4.6               3.1                1.5               0.2
 1                4.6              20.0                1.5               0.2
 2                4.6              20.0               30.0               0.2
 3                4.6              20.0               30.0              40.0
 4               10.0              20.0               30.0              40.0
 5               10.0               3.1               30.0              40.0
 6               10.0               3.1                1.5              40.0
 7               10.0               3.1                1.5               0.2
 8                4.6               3.1                1.5               0.2,)

In [33]:
# permutation 2

inds

array([0, 3, 2, 1], dtype=int64)

- empty -> 0
- 0 -> 3
- 03 -> 2
- 023 -> 1


- empty -> 1
- 1 -> 2
- 12 -> 3
- 123 -> 0

In [34]:
mi[0]

(   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
 0                4.6               3.1                1.5               0.2
 1               10.0               3.1                1.5               0.2
 2               10.0               3.1                1.5              40.0
 3               10.0               3.1               30.0              40.0
 4               10.0              20.0               30.0              40.0
 5                4.6              20.0               30.0              40.0
 6                4.6              20.0               30.0               0.2
 7                4.6              20.0                1.5               0.2
 8                4.6               3.1                1.5               0.2,)

In [37]:
# permutation 3

inds

array([0, 2, 1, 3], dtype=int64)

- empty -> 0
- 0 -> 2
- 02 -> 1
- 012 -> 3


- empty -> 3
- 3 -> 1
- 13 -> 2
- 123 -> 0

In [38]:
mi[0]

(   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
 0                4.6               3.1                1.5               0.2
 1               10.0               3.1                1.5               0.2
 2               10.0               3.1               30.0               0.2
 3               10.0              20.0               30.0               0.2
 4               10.0              20.0               30.0              40.0
 5                4.6              20.0               30.0              40.0
 6                4.6              20.0                1.5              40.0
 7                4.6               3.1                1.5              40.0
 8                4.6               3.1                1.5               0.2,)

In [29]:
# the average prediction for each coalition

outputs = fm.model(*subset_masked_inputs)
outputs

array([0, 0, 1, 2, 2, 2, 0, 0, 0], dtype=int64)

In [39]:
# this is used to add up the marginals for
# each feature

row_values = np.zeros((len(fm),) + outputs.shape[1:])
row_values

array([0., 0., 0., 0.])

In [41]:
inds

array([0, 2, 1, 3], dtype=int64)

In [40]:
# update our SHAP value estimates

i = 0
history_pos = 0
for ind in inds: # forward
    row_values[ind] += outputs[i + 1] - outputs[i]
    i += 1
print(row_values)
history_pos += 1
for ind in inds: # backward
    row_values[ind] += outputs[i] - outputs[i + 1]
    i += 1
print(row_values)
history_pos += 1

[0. 1. 1. 0.]
[0. 1. 3. 0.]
